In [1]:
from utilities.utilities import load_data, get_records_by_region, create_column, finalize_dataframe, get_extreme_values, create_directory_structure, save_table, save_report, pd, assign_quartile, calculate_rank
# settings
region_column_name = 'Region'
table_name = 'security_headers_by_region'
report_name = 'security_headers_by_region'
category = 'security_headers'
column_name_to_results_global = 'Global #'
create_directory_structure()

source_df = load_data('security_headers_checker')

In [2]:
source_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 458 entries, 0 to 457
Data columns (total 16 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   id                                 458 non-null    int64 
 1   region                             458 non-null    object
 2   name                               458 non-null    object
 3   category                           458 non-null    object
 4   url                                458 non-null    object
 5   strict-transport-security          458 non-null    bool  
 6   x-frame-options                    458 non-null    bool  
 7   x-content-type-options             458 non-null    bool  
 8   content-security-policy            458 non-null    bool  
 9   x-permitted-cross-domain-policies  458 non-null    bool  
 10  referrer-policy                    458 non-null    bool  
 11  clear-site-data                    458 non-null    bool  
 12  cross-or

In [3]:
# sanity dataset
headers = [
            'strict-transport-security',
            'x-frame-options',
            'x-content-type-options',
            'content-security-policy',
            'x-permitted-cross-domain-policies',
            'referrer-policy',
            'clear-site-data',
            'cross-origin-embedder-policy',
            'cross-origin-opener-policy',
            'cross-origin-resource-policy',
            'cache-control'
        ]

# Records that are null suffered an error during data collection and are therefore treated as not having security headers.
for header in headers:
    source_df.loc[source_df[header].isna(), header] = False

In [13]:
# Analysis of HEIs with at least one OWASP Recommended Security Headers by region (Pub/Pvt)

# settings
column_to_sort = 'With no Security Headers %'
sort_ascending = False
config = [
    {'table_name': 'security_headers_by_region_public', 'hei_type': 'Public'},
    {'table_name': 'security_headers_by_region_private', 'hei_type': 'Private'}
]
dfs = []
for config_item in config:
    table_name = config_item['table_name']
    hei_type = config_item['hei_type']
    columns_to_display = [region_column_name.title(), column_name_to_results_global]
    analysis_df = get_records_by_region(source_df, hei_type=hei_type)
    # create columns
    
    
    # Column creation with distribuitions of records by amount of security headers by region
    source_df['security_header_count'] = source_df[headers].astype(int).sum(axis=1)

    header_ranges = {
        'no': (0, 0),
        '1': (1, 1),
        '2-3': (2, 3),
        '4-5': (4, 5),
        '6-8': (6, 8),
        '9-10': (9, 10),
        'All': (11, 11)
    }

    for label, (min_headers, max_headers) in header_ranges.items():
        criteria = f'category == "{hei_type}" & {min_headers} <= security_header_count <= {max_headers}'
        create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name=f'With {label} Security Headers', criteria=criteria, columns_to_display=columns_to_display)

    # Finalize dataframe
    analysis_df = finalize_dataframe(dataframe=analysis_df, column_to_sort=column_to_sort, ascending=sort_ascending, columns_to_display=columns_to_display)
    display(analysis_df)
    dfs.append(analysis_df)
    # save to csv
    save_table(analysis_df, category=category, table_name=table_name)



df_public = dfs[0].add_suffix('(pub)')
df_private = dfs[1].add_suffix('(pvt)')
df_public = df_public.rename(columns={'Region(pub)': 'Region'})
df_private = df_private.rename(columns={'Region(pvt)': 'Region'})
df_combined = df_public.merge(df_private, on='Region', how='outer')
df_combined.fillna(0, inplace=True)
# remove columns with # in the name
df_combined = df_combined.loc[:, ~df_combined.columns.str.contains('#')]
#remove columns global
df_combined = df_combined.loc[:, ~df_combined.columns.str.contains('Global')]
# remove '%' from name of columns
df_combined.columns = df_combined.columns.str.replace('%', '')




ranks_columns = []
# Specify the columns to use
columns_to_use = ['With All Security Headers', 'With 9-10 Security Headers', 'With 6-8 Security Headers',
                  'With 4-5 Security Headers']
# Apply the function to calculate the rank
for i, column in enumerate(columns_to_use):
    rank_colum = f'Rank {i}'
    ranks_columns.append(rank_colum)
    df_combined[rank_colum] = df_combined.apply(lambda row: calculate_rank(row, column), axis=1)

#order dataframe by column Rank (from highest to lowest)
df_combined = df_combined.sort_values(by=ranks_columns, ascending=False)
# move just row with 'Total' in column Region to the end of the dataframe. (Use pandas.concat instead of append to avoid duplicates)
df_combined = pd.concat([df_combined[df_combined['Region'] != 'Total'], df_combined[df_combined['Region'] == 'Total']])
# reset index
df_combined.reset_index(drop=True, inplace=True)
# remove column Rank
df_combined.drop(columns=ranks_columns, inplace=True)
#Add a column with the quartile corresponding to the position of the row, that is, considering the total of records -1 (to exclude the total row), if a row is in position 2 it should belong to the first quartile.
df_combined['Quartile'] = df_combined.index.map(lambda rank: assign_quartile(rank, len(df_combined)-1))
# moved column 'Quartile' to the second position
cols = list(df_combined.columns)
cols = [cols[0]] + [cols[-1]] + cols[1:-1]
df_combined = df_combined[cols]

display(df_combined)
save_table(df_combined, category=category, table_name='security_headers_by_region_combined')

,Region,Global #,With no Security Headers #,With no Security Headers %,With 1 Security Headers #,With 1 Security Headers %,With 2-3 Security Headers #,With 2-3 Security Headers %,With 4-5 Security Headers #,With 4-5 Security Headers %,With 6-8 Security Headers #,With 6-8 Security Headers %,With 9-10 Security Headers #,With 9-10 Security Headers %,With All Security Headers #,With All Security Headers %
0,Sachsen,23,9,39.130435,3,13.043478,2,8.695652,6,26.086957,3,13.043478,0,0.000000,0,0.0
1,Bremen,5,1,20.000000,1,20.000000,1,20.000000,1,20.000000,1,20.000000,0,0.000000,0,0.0
2,Schleswig-Holstein,10,2,20.000000,4,40.000000,4,40.000000,0,0.000000,0,0.000000,0,0.000000,0,0.0
3,Nordrhein-Westfalen,42,8,19.047619,5,11.904762,14,33.333333,8,19.047619,7,16.666667,0,0.000000,0,0.0
4,Hessen,22,4,18.181818,2,9.090909,8,36.363636,6,27.272727,2,9.090909,0,0.000000,0,0.0
5,Baden-Württemberg,48,7,14.583333,6,12.500000,23,47.916667,9,18.750000,2,4.166667,1,2.083333,0,0.0
6,Mecklenburg-Vorpommern,7,1,14.285714,0,0.000000,4,57.142857,2,28.571429,0,0.000000,0,0.000000,0,0.0
7,Niedersachsen,21,3,14.285714,2,9.523810,10,47.619048,6,28.571429,0,0.000000,0,0.000000,0,0.0
8,Rheinland-Pfalz,15,2,13.333333,1,6.666667,6,40.000000,6,40.000000,0,0.000000,0,0.000000,0,0.0
9,Brandenburg,10,1,10.000000,4,40.000000,4,40.000000,1,10.000000,0,0.000000,0,0.000000,0,0.0


,Region,Global #,With no Security Headers #,With no Security Headers %,With 1 Security Headers #,With 1 Security Headers %,With 2-3 Security Headers #,With 2-3 Security Headers %,With 4-5 Security Headers #,With 4-5 Security Headers %,With 6-8 Security Headers #,With 6-8 Security Headers %,With 9-10 Security Headers #,With 9-10 Security Headers %,With All Security Headers #,With All Security Headers %
0,Rheinland-Pfalz,6,3,50.000000,1,16.666667,1,16.666667,1,16.666667,0,0.000000,0,0.0,0,0.0
1,Hessen,23,8,34.782609,5,21.739130,9,39.130435,1,4.347826,0,0.000000,0,0.0,0,0.0
2,Berlin,28,7,25.000000,4,14.285714,10,35.714286,6,21.428571,1,3.571429,0,0.0,0,0.0
3,Brandenburg,9,2,22.222222,2,22.222222,2,22.222222,0,0.000000,3,33.333333,0,0.0,0,0.0
4,Baden-Württemberg,24,5,20.833333,5,20.833333,11,45.833333,1,4.166667,2,8.333333,0,0.0,0,0.0
5,Hamburg,11,2,18.181818,3,27.272727,3,27.272727,3,27.272727,0,0.000000,0,0.0,0,0.0
6,Bayern,12,2,16.666667,1,8.333333,5,41.666667,3,25.000000,1,8.333333,0,0.0,0,0.0
7,Nordrhein-Westfalen,25,4,16.000000,9,36.000000,6,24.000000,6,24.000000,0,0.000000,0,0.0,0,0.0
8,Sachsen,7,1,14.285714,0,0.000000,6,85.714286,0,0.000000,0,0.000000,0,0.0,0,0.0
9,Niedersachsen,16,2,12.500000,3,18.750000,3,18.750000,6,37.500000,2,12.500000,0,0.0,0,0.0


,Region,Quartile,With no Security Headers (pub),With 1 Security Headers (pub),With 2-3 Security Headers (pub),With 4-5 Security Headers (pub),With 6-8 Security Headers (pub),With 9-10 Security Headers (pub),With All Security Headers (pub),With no Security Headers (pvt),With 1 Security Headers (pvt),With 2-3 Security Headers (pvt),With 4-5 Security Headers (pvt),With 6-8 Security Headers (pvt),With 9-10 Security Headers (pvt),With All Security Headers (pvt)
0,Baden-Württemberg,1,14.583333,12.500000,47.916667,18.750000,4.166667,2.083333,0.0,20.833333,20.833333,45.833333,4.166667,8.333333,0.0,0.0
1,Brandenburg,1,10.000000,40.000000,40.000000,10.000000,0.000000,0.000000,0.0,22.222222,22.222222,22.222222,0.000000,33.333333,0.0,0.0
2,Bayern,1,2.941176,17.647059,44.117647,23.529412,11.764706,0.000000,0.0,16.666667,8.333333,41.666667,25.000000,8.333333,0.0,0.0
3,Bremen,1,20.000000,20.000000,20.000000,20.000000,20.000000,0.000000,0.0,0.000000,50.000000,50.000000,0.000000,0.000000,0.0,0.0
4,Thüringen,1,0.000000,18.181818,27.272727,36.363636,18.181818,0.000000,0.0,0.000000,0.000000,50.000000,50.000000,0.000000,0.0,0.0
5,Nordrhein-Westfalen,2,19.047619,11.904762,33.333333,19.047619,16.666667,0.000000,0.0,16.000000,36.000000,24.000000,24.000000,0.000000,0.0,0.0
6,Sachsen,2,39.130435,13.043478,8.695652,26.086957,13.043478,0.000000,0.0,14.285714,0.000000,85.714286,0.000000,0.000000,0.0,0.0
7,Niedersachsen,2,14.285714,9.523810,47.619048,28.571429,0.000000,0.000000,0.0,12.500000,18.750000,18.750000,37.500000,12.500000,0.0,0.0
8,Hamburg,2,10.000000,20.000000,40.000000,20.000000,10.000000,0.000000,0.0,18.181818,27.272727,27.272727,27.272727,0.000000,0.0,0.0
9,Hessen,3,18.181818,9.090909,36.363636,27.272727,9.090909,0.000000,0.0,34.782609,21.739130,39.130435,4.347826,0.000000,0.0,0.0


In [ ]:
# Analysis of HEIs with at least one OWASP Recommended Security Headers by region

# settings
column_to_sort = 'Without Security Headers (Public) %'
sort_ascending = True
columns_to_display = [region_column_name, column_name_to_results_global]
analysis_df = get_records_by_region(source_df)

# create columns
# Column creation with distribution of records without any security headers by region
only_public = 'category == "Public"'
only_headers_false = ' & '.join([f"`{h}` == False" for h in headers])
criteria = f"{only_public} & ({only_headers_false})"
create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='Without Security Headers (Public)', criteria=criteria, columns_to_display=columns_to_display)
only_private = 'category == "Private"'
criteria = f"{only_private} & ({only_headers_false})"
create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='Without Security Headers (Private)', criteria=criteria, columns_to_display=columns_to_display)

# Column creation with distribution of records with at least one security header by region
all_true_filter = ' & '.join([f"`{h}` == True" for h in headers])
at_least_one_filter = ' | '.join([f"`{h}` == True" for h in headers])
composite_filter = f"({at_least_one_filter}) & ~({all_true_filter})"
criteria = f"{only_public} & ({composite_filter})"
create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='With At Least One Security Header (Public)', criteria=criteria, columns_to_display=columns_to_display)
criteria = f"{only_private} & ({composite_filter})"
create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='With At Least One Security Header (Private)', criteria=criteria, columns_to_display=columns_to_display)

# Column creation with distribution of records with all security header by region
criteria = f"{only_public} & ({all_true_filter})"
create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='With All Security Headers (Public)', criteria=criteria, columns_to_display=columns_to_display)
criteria = f"{only_private} & ({all_true_filter})"
create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='With All Security Headers (Private)', criteria=criteria, columns_to_display=columns_to_display)

# Finalize dataframe
analysis_df = finalize_dataframe(dataframe=analysis_df, column_to_sort=column_to_sort, ascending=sort_ascending, columns_to_display=columns_to_display)
display(analysis_df)

# save to csv
save_table(analysis_df, category=category, table_name=table_name)


In [ ]:
# Report in latex
report_results = get_extreme_values(analysis_df)

hei_public_without_security_headers = format(report_results.get("Total").get("Without Security Headers (Public) %"), ".2f")
hei_public_with_security_headers = format(report_results.get("Total").get("With At Least One Security Header (Public) %"), ".2f")
hei_public_with_all_security_headers = format(report_results.get("Total").get("With All Security Headers (Public) %"), ".2f")
hei_private_without_security_headers = format(report_results.get("Total").get("Without Security Headers (Private) %"), ".2f")
hei_private_with_security_headers = format(report_results.get("Total").get("With At Least One Security Header (Private) %"), ".2f")
hei_private_with_all_security_headers = format(report_results.get("Total").get("With All Security Headers (Private) %"), ".2f")

report_figure = """
\\begin{figure}[htbp]
    \centering
    \includegraphics[width=0.48\\textwidth]{charts/security_headers_by_region.pdf}
    \caption{Distribution of the use Security Headers by region.}\label{fig:security-headers}
\end{figure}
"""

report = f'{report_figure}\n\n'
report += f'The data presented in Figure~\\ref{{fig:security-headers}} provides an overview of the use of security headers at \glspl{{hei}} in \countryName. According to the data, {hei_public_without_security_headers}\% of the public institutions analyzed have not implemented any security headers on their websites, while {hei_private_without_security_headers}\% of the private institutions analyzed also lack security headers.\n\n'
report += f'On a positive note, {hei_public_with_security_headers}\% of the public institutions analyzed have implemented at least one security header in their websites, and {hei_private_with_security_headers}\% of the private institutions have implemented at least one security header.\n\n'
report += f'Finally, {hei_public_with_all_security_headers}\% of the public institutions analyzed have implemented all the security headers in their websites, and {hei_private_with_all_security_headers}\% of the private institutions have implemented all the security headers.\n\n'
report += f'In terms of regional differences, private institutions in {report_results.get("With At Least One Security Header (Private) %").get("top_regions")[0][0]} ({format(report_results.get("With At Least One Security Header (Private) %").get("top_regions")[0][1], ".2f")}\%), {report_results.get("With At Least One Security Header (Private) %").get("top_regions")[1][0]} ({format(report_results.get("With At Least One Security Header (Private) %").get("top_regions")[1][1], ".2f")}\%), and {report_results.get("With At Least One Security Header (Private) %").get("top_regions")[2][0]} ({format(report_results.get("With At Least One Security Header (Private) %").get("top_regions")[2][1], ".2f")}\%), '
report += f'while public institutions in {report_results.get("With At Least One Security Header (Public) %").get("top_regions")[0][0]} ({format(report_results.get("With At Least One Security Header (Public) %").get("top_regions")[0][1], ".2f")}\%), {report_results.get("With At Least One Security Header (Public) %").get("top_regions")[1][0]} ({format(report_results.get("With At Least One Security Header (Public) %").get("top_regions")[1][1], ".2f")}\%), and {report_results.get("With At Least One Security Header (Public) %").get("top_regions")[2][0]} ({format(report_results.get("With At Least One Security Header (Public) %").get("top_regions")[2][1], ".2f")}\%) have a higher usage of security headers.\n\n'
report += f'In contrast, private institutions in {report_results.get("With At Least One Security Header (Private) %").get("bottom_regions")[0][0]} ({format(report_results.get("With At Least One Security Header (Private) %").get("bottom_regions")[0][1], ".2f")}\%), {report_results.get("With At Least One Security Header (Private) %").get("bottom_regions")[1][0]} ({format(report_results.get("With At Least One Security Header (Private) %").get("bottom_regions")[1][1], ".2f")}\%), and {report_results.get("With At Least One Security Header (Private) %").get("bottom_regions")[2][0]} ({format(report_results.get("With At Least One Security Header (Private) %").get("bottom_regions")[2][1], ".2f")}\%), '
report += f'while public institutions in {report_results.get("With At Least One Security Header (Public) %").get("bottom_regions")[0][0]} ({format(report_results.get("With At Least One Security Header (Public) %").get("bottom_regions")[0][1], ".2f")}\%), {report_results.get("With At Least One Security Header (Public) %").get("bottom_regions")[1][0]} ({format(report_results.get("With At Least One Security Header (Public) %").get("bottom_regions")[1][1], ".2f")}\%), and {report_results.get("With At Least One Security Header (Public) %").get("bottom_regions")[2][0]} ({format(report_results.get("With At Least One Security Headedfddddddddddddddddr (Public) %").get("bottom_regions")[2][1], ".2f")}\%) have a lower usage of security headers.\n\n'

print(report)

# save report to file txt
save_report(report=report, category=category, report_name=report_name)

In [ ]:
# specific security headers by region (Pub/Pvt)

# settings
column_to_sort = 'Without Security Headers %'
sort_ascending = True
config = [
    {'table_name': 'security_headers_by_region_public', 'hei_type': 'Public'},
    {'table_name': 'security_headers_by_region_private', 'hei_type': 'Private'}
]
for config_item in config:
    hei_type = config_item['hei_type']
    for header in headers:
        table_name = f'{header.lower()}_by_region_{hei_type.lower()}'
        report_name = f'{header.lower()}_by_region_{hei_type.lower()}'
        columns_to_display = [region_column_name, column_name_to_results_global]
        analysis_df = get_records_by_region(source_df, hei_type=hei_type)

        # Column creation with distribution of records without any security headers by region
        only_headers_false = ' & '.join([f"`{h}` == False" for h in headers])
        criteria = f'category == "{hei_type}" & ({only_headers_false})'
        create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='Without Security Headers', criteria=criteria, columns_to_display=columns_to_display)
        # Column creation with distribution of records with a specific security header by region
        create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name=f"With {header}", criteria=f'category == "{hei_type}"  & (`{header}` == True)', columns_to_display=columns_to_display)

        # Column creation with distribution of records without a specific security header by region
        true_at_least_one_header_except_current = ' | '.join([f"`{h}` == True" for h in headers if h != header])
        criteria = f"(`{header}` == False) & ({true_at_least_one_header_except_current})"
        create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name=f"Without {header}", criteria=f'category == "{hei_type}" & {criteria}', columns_to_display=columns_to_display)
        display(analysis_df)

        # Finalize dataframe
        analysis_df = finalize_dataframe(dataframe=analysis_df, column_to_sort=column_to_sort, ascending=sort_ascending, columns_to_display=columns_to_display)
        display(analysis_df)
        # save to csv
        save_table(analysis_df, category=category, table_name=table_name)


In [ ]:
# specific security headers by region

# settings
column_to_sort = 'Without Security Headers (Public) %'
sort_ascending = True


for header in headers:
    table_name = f'{header.lower()}_by_region'
    report_name = f'{header.lower()}_by_region'
    columns_to_display = [region_column_name, column_name_to_results_global]
    analysis_df = get_records_by_region(source_df)

    # Column creation with distribution of records without any security headers by region
    only_public = 'category == "Public"'
    only_headers_false = ' & '.join([f"`{h}` == False" for h in headers])
    criteria = f"{only_public} & ({only_headers_false})"
    create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='Without Security Headers (Public)', criteria=criteria, columns_to_display=columns_to_display)
    only_private = 'category == "Private"'
    criteria = f"{only_private} & ({only_headers_false})"
    create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='Without Security Headers (Private)', criteria=criteria, columns_to_display=columns_to_display)

    # Column creation with distribution of records with a specific security header by region
    create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name=f"With {header} (Public)", criteria=f"{only_public} & (`{header}` == True)", columns_to_display=columns_to_display)
    create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name=f"With {header} (Private)", criteria=f"{only_private} & (`{header}` == True)", columns_to_display=columns_to_display)

    # Column creation with distribution of records without a specific security header by region
    true_at_least_one_header_except_current = ' | '.join([f"`{h}` == True" for h in headers if h != header])
    criteria = f"(`{header}` == False) & ({true_at_least_one_header_except_current})"
    create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name=f"Without {header} (Public)", criteria=f"{only_public} & {criteria}", columns_to_display=columns_to_display)
    create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name=f"Without {header} (Private)", criteria=f"{only_private} & {criteria}", columns_to_display=columns_to_display)
    display(analysis_df)

    # Finalize dataframe
    analysis_df = finalize_dataframe(dataframe=analysis_df, column_to_sort=column_to_sort, ascending=sort_ascending, columns_to_display=columns_to_display)
    display(analysis_df)
    # save to csv
    save_table(analysis_df, category=category, table_name=table_name)

    # save to latex
    # report
    report_results = get_extreme_values(analysis_df)
    hei_private_with_specific_header = format(report_results.get("Total").get(f'With {header} (Private) %'), ".2f")
    hei_private_without_specific_header = format(report_results.get("Total").get(f"Without {header} (Private) %"), ".2f")
    hei_public_with_specific_header = format(report_results.get("Total").get(f"With {header} (Public) %"), ".2f")
    hei_public_without_specific_header = format(report_results.get("Total").get(f"Without {header} (Public) %"), ".2f")

    report_figure = f"""
    \\begin{{figure}}[htbp]
        \centering
        \includegraphics[width=0.48\\textwidth]{{charts/{table_name}.pdf}}
        \caption{{Distribution of {header} header usage by region.}}\label{{fig:{header}}}
    \end{{figure}}
    """

    report = f'{report_figure}\n\n'
    report += f'The data presented in Figure~\\ref{{fig:{header}}} provides an overview of the use of {header} security headers at \glspl{{hei}} in \countryName. According to the data, {hei_public_without_specific_header}\% of the public institutions analyzed have not implemented {header} security header on their websites, while {hei_private_without_specific_header}\% of the private institutions analyzed also lack {header} security header.\n\n'
    report += f'On a positive note, {hei_public_with_specific_header}\% of the public institutions analyzed have implemented {header} security headers on their websites, while {hei_private_with_specific_header}\% of the private institutions analyzed also implemented that security header.\n\n'
    report += f'In terms of regional differences, private institutions in {report_results.get(f"With {header} (Private) %").get("top_regions")[0][0]} ({format(report_results.get(f"With {header} (Private) %").get("top_regions")[0][1], ".2f")}\%), {report_results.get(f"With {header} (Private) %").get("top_regions")[1][0]} ({format(report_results.get(f"With {header} (Private) %").get("top_regions")[1][1], ".2f")}\%), and {report_results.get(f"With {header} (Private) %").get("top_regions")[2][0]} ({format(report_results.get(f"With {header} (Private) %").get("top_regions")[2][1], ".2f")}\%) have a higher usage of {header} header, '
    report += f'while public institutions in {report_results.get(f"With {header} (Public) %").get("top_regions")[0][0]} ({format(report_results.get(f"With {header} (Public) %").get("top_regions")[0][1], ".2f")}\%), {report_results.get(f"With {header} (Public) %").get("top_regions")[1][0]} ({format(report_results.get(f"With {header} (Public) %").get("top_regions")[1][1], ".2f")}\%), and {report_results.get(f"With {header} (Public) %").get("top_regions")[2][0]} ({format(report_results.get(f"With {header} (Public) %").get("top_regions")[2][1], ".2f")}\%) have a higher usage of {header} security header.\n\n'
    report += f'In contrast, private institutions in {report_results.get(f"With {header} (Private) %").get("bottom_regions")[0][0]} ({format(report_results.get(f"With {header} (Private) %").get("bottom_regions")[0][1], ".2f")}\%), {report_results.get(f"With {header} (Private) %").get("bottom_regions")[1][0]} ({format(report_results.get(f"With {header} (Private) %").get("bottom_regions")[1][1], ".2f")}\%), and {report_results.get(f"With {header} (Private) %").get("bottom_regions")[2][0]} ({format(report_results.get(f"With {header} (Private) %").get("bottom_regions")[2][1], ".2f")}\%), '
    report += f'while public institutions in {report_results.get(f"With {header} (Public) %").get("bottom_regions")[0][0]} ({format(report_results.get(f"With {header} (Public) %").get("bottom_regions")[0][1], ".2f")}\%), {report_results.get(f"With {header} (Public) %").get("bottom_regions")[1][0]} ({format(report_results.get(f"With {header} (Public) %").get("bottom_regions")[1][1], ".2f")}\%), and {report_results.get(f"With {header} (Public) %").get("bottom_regions")[2][0]} ({format(report_results.get(f"With {header} (Public) %").get("bottom_regions")[2][1], ".2f")}\%) have a lower usage of security headers.\n\n'
    # save report to file
    print(report)
    save_report(report=report, category=category, report_name=report_name)